# 07 Label budget vs method choice
Hypothesis: best method depends on target label budget.

### Expected Outcome
Feature extraction should excel in very low-label settings; gradual unfreezing should improve with larger budgets.

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
while ROOT != ROOT.parent and not (ROOT / 'src').is_dir():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / 'src'))

from utils.notebook_transfer import TransferNotebookLab

LAB = TransferNotebookLab.from_root(ROOT)
FAST_DEV_RUN = False

In [ ]:
methods = ['scratch', 'feature_extraction', 'gradual_unfreeze']
budgets = [10, 20] if FAST_DEV_RUN else [20, 40, 80, 120]
rows = []

for budget in budgets:
    LAB.run(
        base_name='transfer_core_related.yaml',
        notebook_tag=f'07_budget_{budget}',
        fast_dev_run=FAST_DEV_RUN,
        overrides={
            'data': {'target_train_per_class': budget},
            'methods': methods,
        },
    )
    for method in methods:
        df = LAB.read_method(method)
        rows.append({
            'budget': budget,
            'method': method,
            'best_target_acc': float(df['target_test_acc'].max()),
            'final_target_acc': float(df['target_test_acc'].iloc[-1]),
            'final_feature_drift': float(df['feature_drift'].iloc[-1]),
        })

results = pd.DataFrame(rows)
results.sort_values(['budget', 'best_target_acc'], ascending=[True, False])

In [ ]:
fig, ax = plt.subplots(figsize=(6.8, 3.8))
for method, df in results.groupby('method'):
    ax.plot(df['budget'], df['best_target_acc'], marker='o', label=method)
ax.set_title('Best target accuracy by label budget')
ax.set_xlabel('target labels per class')
ax.set_ylabel('best_target_acc')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '07_budget_vs_best_target_acc.png')

fig, ax = plt.subplots(figsize=(6.8, 3.8))
for method, df in results.groupby('method'):
    ax.plot(df['budget'], df['final_feature_drift'], marker='o', label=method)
ax.set_title('Final feature drift by label budget')
ax.set_xlabel('target labels per class')
ax.set_ylabel('final_feature_drift')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '07_budget_vs_feature_drift.png')

In [ ]:
results.pivot(index='budget', columns='method', values='best_target_acc')

### Interpretation
Method selection should be budget-aware, not fixed globally.